In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/retinopaty/data'

In [ ]:
import os, shutil

In [ ]:
train_dir = '/content/drive/MyDrive/retinopaty/data/training'

In [ ]:
validation_dir = '/content/drive/MyDrive/retinopaty/data/validation'

In [ ]:
test_dir = '/content/drive/MyDrive/retinopaty/data/test'

In [ ]:
Mild_DR_dir = os.path.join(train_dir, 'Mild DR')
No_DR_dir = os.path.join(train_dir, 'No DR')
Moderate_DR_dir = os.path.join(train_dir, 'Moderate DR')
Severe_DR_dir = os.path.join(train_dir, 'Severe DR')
Proliferative_DR_dir = os.path.join(train_dir, 'Proliferative DR')

In [ ]:
print('total training Mild DR images:', len(os.listdir(Mild_DR_dir)))
print('total training No DR images:', len(os.listdir(No_DR_dir)))
print('total training Moderate DR images:', len(os.listdir(Moderate_DR_dir)))
print('total training Severe DR images:', len(os.listdir(Severe_DR_dir)))
print('total training Proliferative DR images:', len(os.listdir(Proliferative_DR_dir)))


total training Mild DR images: 437
total training No DR images: 427
total training Moderate DR images: 438
total training Severe DR images: 224
total training Proliferative DR images: 155


In [ ]:
from keras import layers
import tensorflow as tf
from keras import models
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='categorical')

Found 1681 images belonging to 5 classes.
Found 440 images belonging to 5 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20, 5)


In [ ]:
import numpy as np
nb_train_steps = int(np.ceil(len(os.listdir(Mild_DR_dir) + os.listdir(No_DR_dir) + os.listdir(Moderate_DR_dir) + os.listdir(Severe_DR_dir) + os.listdir(Proliferative_DR_dir))/20))
print(nb_train_steps)
nb_test_steps = 440/20
print(nb_test_steps)

85
22.0


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=nb_test_steps)

Epoch 1/20
85/85 [==============================] - 634s 7s/step - loss: 1.5874 - accuracy: 0.2281 - val_loss: 1.5290 - val_accuracy: 0.2523
Epoch 2/20
85/85 [==============================] - 14s 164ms/step - loss: 1.5466 - accuracy: 0.2662 - val_loss: 1.5411 - val_accuracy: 0.2750
Epoch 3/20
85/85 [==============================] - 14s 164ms/step - loss: 1.5382 - accuracy: 0.2638 - val_loss: 1.5371 - val_accuracy: 0.2523
Epoch 4/20
85/85 [==============================] - 14s 163ms/step - loss: 1.5215 - accuracy: 0.2864 - val_loss: 1.5311 - val_accuracy: 0.2909
Epoch 5/20
85/85 [==============================] - 14s 163ms/step - loss: 1.5358 - accuracy: 0.2751 - val_loss: 1.5333 - val_accuracy: 0.2705
Epoch 6/20
85/85 [==============================] - 14s 162ms/step - loss: 1.5350 - accuracy: 0.2710 - val_loss: 1.5331 - val_accuracy: 0.2500
Epoch 7/20
85/85 [==============================] - 14s 163ms/step - loss: 1.5346 - accuracy: 0.2965 - val_loss: 1.5326 - val_accuracy: 0.2341
E

In [ ]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=15,
                                   vertical_flip=True,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5]
                                   )

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(validation_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1681 images belonging to 5 classes.
Found 440 images belonging to 5 classes.


In [ ]:
for data_batch, labels_batch in training_set:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 5)


In [ ]:
print(batch_size)

In [ ]:
nb_train_steps = int(np.ceil(len(train_dir)/batch_size))
history = model.fit(training_set,
                    epochs=10,
                    steps_per_epoch=53,
                         validation_data = test_set
                         )

Epoch 1/10
53/53 [==============================] - 23s 417ms/step - loss: 1.5691 - accuracy: 0.2519 - val_loss: 1.5323 - val_accuracy: 0.2500
Epoch 2/10
53/53 [==============================] - 22s 412ms/step - loss: 1.5419 - accuracy: 0.2620 - val_loss: 1.5316 - val_accuracy: 0.2568
Epoch 3/10
53/53 [==============================] - 22s 412ms/step - loss: 1.5410 - accuracy: 0.2690 - val_loss: 1.5365 - val_accuracy: 0.2409
Epoch 4/10
53/53 [==============================] - 22s 412ms/step - loss: 1.5387 - accuracy: 0.2573 - val_loss: 1.5333 - val_accuracy: 0.2500
Epoch 5/10
53/53 [==============================] - 22s 414ms/step - loss: 1.5401 - accuracy: 0.2715 - val_loss: 1.5305 - val_accuracy: 0.2432
Epoch 6/10
53/53 [==============================] - 22s 412ms/step - loss: 1.5507 - accuracy: 0.2503 - val_loss: 1.5289 - val_accuracy: 0.2500
Epoch 7/10
53/53 [==============================] - 22s 412ms/step - loss: 1.5498 - accuracy: 0.2586 - val_loss: 1.5263 - val_accuracy: 0.3000

In [ ]:
print()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=16,
                                                        class_mode='categorical')

Found 1681 images belonging to 5 classes.
Found 440 images belonging to 5 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 224, 224, 3)
labels batch shape: (32, 5)


In [ ]:
history = model.fit(
    training_set,
    steps_per_epoch = 150,
    # steps_per_epoch = int( np.ceil(x_train.shape[0] / batch_size) ),
    epochs=20,
    validation_data=test_set,
    validation_steps=62)

Epoch 1/20
150/150 [==============================] - 22s 146ms/step - loss: 0.5005 - accuracy: 0.8000 - val_loss: 0.5004 - val_accuracy: 0.8000


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
from keras.preprocessing import image
fnames = [os.path.join(Mild_DR_dir, fname) for
          fname in os.listdir(Mild_DR_dir)]
img_path = fnames[3]
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(image.array_to_img(batch[0]))
  i += 1
  if i % 4 == 0:
    break
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')
history = model.fit_generator(train_generator,
                              steps_per_epoch=10,
                              epochs=20,
                              validation_data=validation_generator,
                              validation_steps=50)

In [ ]:
len(model.trainable_variables)

In [ ]:
initial_epochs = 20
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_generator, steps = validation_steps)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=validation_generator)
#Fitting the model in a CNN

In [ ]:
import matplotlib.pyplot as plt
accuracy= history.history['accuracy']
val_accuracy= history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

In [ ]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
# import pydot
from sklearn.metrics import *
from sklearn.model_selection import *
import tensorflow.keras.backend as K

model = Sequential()
model.add(MobileNetV2(input_shape=(264,264,3),include_top=True,weights=None))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
# Creating an output layer
model.add(Dense(units= 5, activation='softmax'))

In [ ]:
img_size=(160,160)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(160, 160),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 1681 images belonging to 5 classes.
Found 440 images belonging to 5 classes.


In [ ]:
IMG_SHAPE = img_size + (3,)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(5)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1280)
(32, 5)


In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input


In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0     

In [ ]:
len(model.trainable_variables)


2

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_generator)

14/14 [==============================] - 4s 212ms/step - loss: 0.9220 - accuracy: 0.2672


In [ ]:
history = model.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator)

Epoch 1/20
53/53 [==============================] - 17s 276ms/step - loss: 0.4941 - accuracy: 0.2706 - val_loss: 0.4815 - val_accuracy: 0.2773
Epoch 2/20
53/53 [==============================] - 14s 257ms/step - loss: 0.5020 - accuracy: 0.2543 - val_loss: 0.4815 - val_accuracy: 0.2545
Epoch 3/20
53/53 [==============================] - 14s 259ms/step - loss: 0.4949 - accuracy: 0.2480 - val_loss: 0.4818 - val_accuracy: 0.2500
Epoch 4/20
53/53 [==============================] - 14s 262ms/step - loss: 0.4984 - accuracy: 0.2394 - val_loss: 0.4815 - val_accuracy: 0.2773
Epoch 5/20
53/53 [==============================] - 14s 258ms/step - loss: 0.5040 - accuracy: 0.2573 - val_loss: 0.4814 - val_accuracy: 0.2818
Epoch 6/20
53/53 [==============================] - 14s 259ms/step - loss: 0.4989 - accuracy: 0.2588 - val_loss: 0.4814 - val_accuracy: 0.2545
Epoch 7/20
53/53 [==============================] - 14s 258ms/step - loss: 0.4962 - accuracy: 0.2428 - val_loss: 0.4814 - val_accuracy: 0.2500

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])
  fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(training_set,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=test_set)

Number of layers in the base model:  154


NameError: ignored